# Creation of dataframe for gravity model

- join on cityname
- create city dataframe (nodes)
- create separate city-pairs dataframes (basically edges)
- calculate distances

Add summary of what this script does

## Required Input
This notebook requires input not available in this repository, namely `euro-global-map-shp/data/FullEurope/BuiltupP.shp`. This dataset can be obtained from [https://www.mapsforeurope.org/datasets/euro-global-map](https://www.mapsforeurope.org/datasets/euro-global-map). 

Furthermore it requires QGIS (or equivalent) for the addition of one of the datapoints.

## Generated output

`../../input/maps/nodes.shp`

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import re
from tqdm.notebook import tqdm
import unidecode
from collections import Counter
from shapely.geometry import Point, LineString, Polygon

In [2]:
INDIR = "../../input"
OUTDIR = "../../output"
DATADIR = "../../../../data" # path/to/non/repo/files
FILE_cities = "List_of_cities_300k.csv"
FILE_coords = "euro-global-map-shp/data/FullEurope/BuiltupP.shp" #path/to/eucoordinates/shapefile

In [3]:
# load citylist
list_of_cities = pd.read_csv(os.path.join(INDIR, FILE_cities), sep=";")
list_of_cities.head()

,id_MUA,Mua,Mua_en,Mua_fr,SizeMUA1k,EU15,Code_Country,NUTS_1,NUTS_2,NUTS_3,...,PUR,Supra_poly_fua,PIA,Name_supra_poly_fua,Poly_fua,id_poly_fua,Name_poly_fua,SizeHinterland1k,GDP_per_capita,Dummy_Capital
0,FR00810,Paris,Paris,Paris,9591,1,FR,FR1,FR10,FR101,...,,0,PIA_Paris,99,0,,,1201,44,1
1,UK01886,London,London,Londres,8256,1,UK,UKI,UKI1,UKI11,...,,0,PIA_London,99,0,,,2752,45,1
2,ES00540,Madrid,Madrid,Madrid,4955,1,ES,ES3,ES30,ES300,...,,0,PIA_Madrid,99,0,,,308,29,1
3,DE00219,Berlin,Berlin,Berlin,3776,1,DE,DE3,DE30,DE300,...,,0,PIA_Berlin,99,0,,,240,22,1
4,IT01156,Milano,Milan,Milan,3698,1,IT,ITC,ITC4,ITC45,...,Milano,0,PIA_Milano,99,1,IT03,Milano metropolitan area,604,35,0


## Select relevant coordinates

In [4]:
# load data with city coordinates
built_up = gpd.read_file(os.path.join(DATADIR, FILE_coords))

In [5]:
built_up.shape

(72846, 18)

In [6]:
built_up.head()

,OBJECTID,FCsubtype,inspireId,beginLifes,F_CODE,ICC,NAMN1,NAMN2,NAMA1,NAMA2,NLN1,NLN2,PPL,PP1,PP2,USE,PopulatedP,geometry
0,1,1,_EG.EGM.BuiltupP:40c2730a-3a4b-474b-8f55-615cd...,20220125104612,AL020,MD,Cernoleuca,UNK,Cernoleuca,UNK,rum,UNK,1771,-32768,-32768,3,N.MD.BUILTUP.000823,POINT (27.56031 48.31269)
1,2,1,_EG.EGM.BuiltupP:f90e3db3-2ef5-4cf7-95ba-2f99f...,20220125104612,AL020,MD,Mo?ana,UNK,Mosana,UNK,rum,UNK,1630,-32768,-32768,3,N.MD.BUILTUP.000825,POINT (27.68990 48.32328)
2,3,1,_EG.EGM.BuiltupP:d947c478-818e-41ed-91a2-0aa79...,20220125104612,AL020,MD,Gribova,UNK,Gribova,UNK,rum,UNK,2101,-32768,-32768,3,N.MD.BUILTUP.000831,POINT (27.93089 48.01388)
3,4,1,_EG.EGM.BuiltupP:0f9b7f83-5249-4030-9933-417ed...,20220125104612,AL020,MD,Chirca,UNK,Chirca,UNK,rum,UNK,1704,-32768,-32768,3,N.MD.BUILTUP.000838,POINT (29.10819 46.92171)
4,5,1,_EG.EGM.BuiltupP:0e4be3d7-1937-4614-a7fd-f6bd2...,20220125104612,AL020,MD,Delac?u,UNK,Delacau,UNK,rum,UNK,2126,-32768,-32768,3,N.MD.BUILTUP.000840,POINT (29.30339 47.09902)


`BuiltUpP.shp` contains 4 different name variables. `NAMN1`, `NAMN2`, `NAMA1`, and `NAMA2`. The first two are names in up to two national languages. The latter are transliterated versions of those names into ASCII characters. 

In [7]:
muas = list(list_of_cities.Mua)

In [8]:
# create a dictionary of toponyms and their country codes
mua_dict = dict()

for index, row in list_of_cities.iterrows():
    if row['Code_Country'] == 'UK': 
        mua_dict[row['Mua']] = 'GB'
    else: 
        mua_dict[row['Mua']] = row['Code_Country']
    

### Names with issues:

At least some of these non matches are combined cities. For these it is probably easiest to choose the coordinates of one. (although i guess the best option would be to calculate the midpoint between the combined cities and then take that);

Another problem is that unidecode replaces 'ü' with 'u' while the `NAMA1` variable lists this with 'ue'. Same for 'oe'.

Den Haag --> 's-Gravenhage (den Haag listed as NAMA2)

**Manual edits**

Antwerp listed with its dutch/flemish name Antwerpen. Brussels as Brussel

Don't understand what is happening to Lyon and Marseille. (listed only with arrondissement numbers in name, e.g. 'Lyon 1er Arrondissement')

error for Gent (negative population number)  
Sofia should be done manually as well (other Sofias are present but Sofia, BU is listed as 'S?fiya')

Stoke --> Stoke-on-Trent
Belfast cause country code does not match

Plovdiv --> Pl?vdiv

Wuppertal not in the dataset. 

suggested process:
1. un-decoded (so with accents). match to NAMA1 first. 
2. Then new list with left over cities, match to NAMN1 (for the umlauts).
3. left-over match to NAMN2
4. Manually match remaining cities




### First Match
Match the cities in the built_up dataset with the cities in the CITYNET network by confirming that name and country match.

In [9]:
cities = []
for index, row in built_up.iterrows(): 
    if row['NAMA1'] in mua_dict and row['ICC'] == mua_dict[row['NAMA1']]:
        cities.append(row['NAMA1'])
    elif row['NAMN1'] in mua_dict and row['ICC'] == mua_dict[row['NAMN1']]:
        cities.append(row['NAMN1'])
    elif row['NAMN2'] in mua_dict and row['ICC'] == mua_dict[row['NAMN2']]: 
        cities.append(row['NAMN2'])

In [10]:
# check how many cities have been matched
len(cities)

137

In [11]:
# check if any cities have been matched more than once
city_counts = Counter(cities) 
for key in city_counts:
    if city_counts[key] > 1:
        print(key)

Bremen
Leeds


In [12]:
# cities not automatically matched
leftovers = []
for city in muas:
    if city not in cities:
        leftovers.append(city)

In [13]:
print(leftovers)

['Brussels', 'Kobenhavn', 'Lyon', 'Sofia', 'Essen-Oberhausen', 'Marseille', 'Antwerp', 'Bochum-Herne', 'Gelsenkirchen-Bottrop', 'Belfast', 'Palma de Mallorca', 'Wuppertal', 'Castellammare di Stabia-Torre Annunziata', 'Plovdiv', 'Alicanta', 'La Coruna']


Matched: 135 cities  
False Match: Bremen and Leeds duplicates

Cities not matched:  
'Brussels', 'Kobenhavn', 'Lyon', 'Sofia', 'Essen-Oberhausen', 'Marseille', 'Antwerp', 'Bochum-Herne', 'Gelsenkirchen-Bottrop', 'Belfast', 'Palma de Mallorca', 'Wuppertal', 'Castellammare di Stabia-Torre Annunziata', 'Plovdiv', 'Alicanta', 'La Coruna'

These cities remain unmatched for a variety of issues.
* language differences (e.g. Brussels, Alicante)
* different name (e.g. Marseille, Mallorca, A Coruna)
* different transliteration (e.g.Plovdiv, Kobenhavn, Sofia)
* MUA is made up of multiple cities (e.g. Castellammare di Stabia-Torre Annunziata)
* Belfast isn't matched cause the country codes don't match (the coordinate dataset uses a separate CC for northern ireland)

One of the cities has no match because it is not present in the coordinate dataset: Wuppertal

Considering the small size of the unmatched city list. A dictionary was created manually to match the diverging names in the two datasets

In [14]:
# dictionary of non matching names
manual_names_v = {"Pl?vdiv":"Plovdiv", 
                "S?fiya":"Sofia", 
                "Koebenhavn": "Kobenhavn", 
                "Brussel": "Brussels", 
                "Antwerpen": "Antwerp",
                "A Coruna": "La Coruna", 
                "Alacant/Alicante": "Alicanta", 
                "Lyon 1er Arrondissement": "Lyon", 
                "Marseille 1er Arrondissement": "Marseille", 
                "Mallorca": "Palma de Mallorca"}

In [15]:
# add first one of the compound cities to the dictionary
for city in leftovers:
    if '-' in city:
        manual_names_v[city.split('-')[0]] = city

In [16]:
# full dictionary
manual_names_v

{'Pl?vdiv': 'Plovdiv',
 'S?fiya': 'Sofia',
 'Koebenhavn': 'Kobenhavn',
 'Brussel': 'Brussels',
 'Antwerpen': 'Antwerp',
 'A Coruna': 'La Coruna',
 'Alacant/Alicante': 'Alicanta',
 'Lyon 1er Arrondissement': 'Lyon',
 'Marseille 1er Arrondissement': 'Marseille',
 'Mallorca': 'Palma de Mallorca',
 'Essen': 'Essen-Oberhausen',
 'Bochum': 'Bochum-Herne',
 'Gelsenkirchen': 'Gelsenkirchen-Bottrop',
 'Castellammare di Stabia': 'Castellammare di Stabia-Torre Annunziata'}

In [17]:
# reverse of dictionary
manual_names_r = {value: key for key, value in manual_names_v.items()}

In [18]:
# add new spellings to the original city : country code dictionary
for key in manual_names_v:
    # create new mua_dict key from the name dictionary and gives it the same country code as the corresponding name
    mua_dict[key] = mua_dict[manual_names_v[key]]

### Match with this new dictionary

In [ ]:
cities = []
for index, row in built_up.iterrows(): 
    if row['NAMA1'] in mua_dict and row['ICC'] == mua_dict[row['NAMA1']]:
        cities.append(tuple(row))
    elif row['NAMN1'] in mua_dict and row['ICC'] == mua_dict[row['NAMN1']]:
        cities.append(tuple(row))
    elif row['NAMN2'] in mua_dict and row['ICC'] == mua_dict[row['NAMN2']]: 
        cities.append(tuple(row))
    # don't require country code match for Belfast (there is only one Belfast in the dataset)
    elif row['NAMA1'] == "Belfast": 
        cities.append(tuple(row))

In [ ]:
df = pd.DataFrame(cities, columns = built_up.columns)
df.shape

In [ ]:
df.head()

In [ ]:
# exclude the duplicate Bremen and Leeds
# Wrong BREMEN 27573
# Wrong LEEDS 63132
duplicate_IDs = [27573, 63132]

# done this way because that way the projection metadata is preserved
gdf = built_up[(~built_up.OBJECTID.isin(duplicate_IDs)) & (built_up.OBJECTID.isin(df.OBJECTID))]

In [ ]:
# save as shapefile (outside this repo)
fp = os.path.join(DATADIR, "city_coordinates.shp")
gdf.to_file(fp)

### Manual Addition
Wuppertal is not in the EGM dataset, and was added manually in QGIS (3.16.12 Hannover) based on OpenStreetMap data.

- Loaded the just created `city_coordinates.shp` in QGIS . 
- Used OSM plugin and overlaid it. Toggled editing then `edit` > `add point feature`
- Added point object to city_coordinates layer. And added 'Wuppertal' for 'NAMN1', and 'DE' for 'ICC'.
- Next `save layer edits` and untoggled editting.
- Finally right-clicked city_coordinates layer > `Export` > `Save Features as` > "city_coordinates_complete.shp"

In [ ]:
# load file with wuppertal coordinates also included
coords = gpd.read_file(os.path.join(DATADIR, "city_coordinates_complete.shp"))

In [ ]:
coords.crs

In [ ]:
coords.head()

## Create Cities dataframe

following columns: Mua, Mua_en, Mua_fr, population, country_code, geometry, (dummies: fr_dum, en_dum, additional disambiguation dummies).  



In [ ]:
df = list_of_cities.copy()
df = df[['id_MUA', 'Mua', 'Mua_en', 'Mua_fr', 'SizeMUA1k', 'Code_Country']]
df.head()

In [ ]:
# change the names in coords so that they match the names in the cities dataframe
coords.replace({'NAMA1': manual_names_v}, inplace=True)

In [ ]:
output_df = []
for index, row in coords.iterrows(): 
    for index2, row2 in df.iterrows(): 
        if row['NAMA1'] == row2['Mua']:
            new_row = list(row2[['id_MUA', 'Mua', 'Mua_en', 'Mua_fr', 'SizeMUA1k', 'Code_Country']])
            new_row.append(row.geometry)
            output_df.append(tuple(new_row))
        elif row['NAMN1'] == row2['Mua']:
            new_row = list(row2[['id_MUA', 'Mua', 'Mua_en', 'Mua_fr', 'SizeMUA1k', 'Code_Country']])
            new_row.append(row.geometry)
            output_df.append(tuple(new_row))
        elif row['NAMN2'] == row2['Mua']: 
            new_row = list(row2[['id_MUA', 'Mua', 'Mua_en', 'Mua_fr', 'SizeMUA1k', 'Code_Country']])
            new_row.append(row.geometry)
            output_df.append(tuple(new_row))

In [ ]:
# transform to GeoDataFrame
output_df = gpd.GeoDataFrame(output_df, columns = ['id_MUA', 'Mua', 'Mua_en', 'Mua_fr', 'SizeMUA1k', 'Code_Country', 'geometry'], crs = coords.crs)

In [ ]:
output_df.plot()

In [ ]:
output_df.crs

In [ ]:
# rename columns to avoid name being cut off when saving
output_df.rename(columns = {'Code_Country': 'CC', 'SizeMUA1k': 'POP', 'Mua_en': 'MUA'}, inplace = True)
output_df.drop(['Mua_fr', 'Mua', 'id_MUA'], axis = 1, inplace = True)

In [ ]:
output_df.head()

In [ ]:
## if you want to save a version of the coordinate shapefile without the dummy values
# fp = os.path.join(INDIR, "maps/city_coordinates.shp")
# output_df.to_file(fp)

### European area zone dummies
*  Central and Eastern Europe
*  Northern Europe
*  Southern Europe
*  Western Europe  

Categorisation derived from EUROVOC:  
[https://eur-lex.europa.eu/browse/eurovoc.html?params=72,7206,914#arrow_914](https://eur-lex.europa.eu/browse/eurovoc.html?params=72,7206,914#arrow_914)

Only countries included in our dataset are listed here, of course more countries belong to these categories. 


| **Central / East** | **West**         | **North** | **South** |
|--------------------|------------------|-----------|-----------|
| Bulgaria           | Austria          | Denmark   | Greece    |
| Czech Republic     | Belgium          | Estonia   | Italy     |
| Hungary            | France           | Finland   | Malta     |
| Poland             | Germany          | Latvia    | Portugal  |
| Romania            | Ireland          | Lithuania | Spain     |
| Slovakia           | Netherlands      | Norway    |       -   |
|       -            | Switzerland      | Sweden    |       -   |
|       -            | (United Kingdom) |     -     |       -   |



In [ ]:
set(output_df.CC)

In [ ]:
west = ['AT', 'BE', 'FR', 'DE', 'IE', 'NL', 'CH', 'UK']
south =  ['GR', 'IT', 'MT', 'PT', 'ES']
north = ['DK', 'EE', 'FI', 'LV', 'LT', 'NO', 'SE']
central_east = ['PL', 'RO', 'HU', 'CZ', 'BG', 'SK']

len(west) + len(south) + len(north) + len(central_east)

In [ ]:
output_df['WEST'] = np.where((output_df['CC'].isin(west)), 1, 0)
output_df['SOUTH'] = np.where((output_df['CC'].isin(south)), 1, 0)
output_df['NORTH'] = np.where((output_df['CC'].isin(north)), 1, 0)
output_df['CEAST'] = np.where((output_df['CC'].isin(central_east)), 1, 0)

In [ ]:
def region_lab(row): 
    if row['WEST'] == 1: 
        region = 'west'
    elif row['CEAST'] == 1:
        region = 'central_east'
    elif row['NORTH'] == 1: 
        region = 'north'
    else: 
        region = 'south'
    return region

In [ ]:
output_df['REGION'] = output_df.apply(region_lab, axis = 1)

In [ ]:
output_df.head()

In [ ]:
output_df[output_df['SOUTH'] == 0]

### Language Sphere dummies

Add language sphere dummies (1 if the city is English or French speaking).

In [ ]:
# create english language sphere dummy based on country code
output_df['EN_DUM'] = np.where((output_df['CC']== 'UK') | (output_df['CC'] == 'IE'), 1, 0)

In [ ]:
# French-speaking cities in countries which speak french only in certain regions
french_sphere = ['Charleroi', 'Liège', 'Brussels', 'Geneva']

# french dummy CC French or city from previous list
fr_dum = []
for i, row in output_df.iterrows(): 
    if row['MUA'] in french_sphere:
        fr_dum.append(1)
    elif row['CC'] == 'FR':
        fr_dum.append(1)
    else:
        fr_dum.append(0)

# add to dataframe
output_df['FR_DUM'] = fr_dum

In [ ]:
output_df.head()

In [ ]:
fp = os.path.join(INDIR, "maps/nodes.shp")
# output_df.to_file(fp)